Предположиние:
- На каждой страницу фиксированое количество segment_id
- Некторые id не активны, однако присутствуют в теле скрипта

Цель: получить segment id, их соответсвие группам.
- Проверить разнгообразие

In [1]:
from bs4 import BeautifulSoup as BS
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import (TimeoutException,
                                        ElementNotVisibleException, 
                                        ElementNotSelectableException, 
                                        NoSuchElementException, 
                                        ElementNotInteractableException, 
                                        StaleElementReferenceException, 
                                        ElementClickInterceptedException, 
                                        InvalidSelectorException, 
                                        WebDriverException,
                                        NoSuchWindowException
                                        )
from urllib3.exceptions import NewConnectionError, MaxRetryError
from typing import NamedTuple
from zipfile import ZipFile, ZIP_DEFLATED
import os, signal, sys, psutil
import re
from time import sleep as time_sleep, time
from datetime import datetime
from collections import deque
import pickle
from tqdm import tqdm
from itertools import chain
import pandas as pd
import numpy as np
import logging
from pathlib import Path
import requests
tqdm.pandas()
requests.packages.urllib3.disable_warnings()
from urllib.parse import urlparse
from urllib.parse import parse_qs
import json
from requests.exceptions import ConnectTimeout

In [3]:
# получение пользовательского логгера и установка уровня логирования
short_logger = logging.getLogger('short_logger')
short_logger.setLevel(logging.INFO)

# настройка обработчика и форматировщика в соответствии с нашими нуждами
short_handler = logging.FileHandler("data/short_logger.log", encoding='utf-8', mode='a')
short_formatter = logging.Formatter("%(asctime)s %(levelname)s %(funcName)s %(message)s")

# добавление форматировщика к обработчику 
short_handler.setFormatter(short_formatter)
# добавление обработчика к логгеру
short_logger.addHandler(short_handler)

# получение пользовательского логгера и установка уровня логирования
long_logger = logging.getLogger('long_logger')
long_logger.setLevel(logging.WARNING)

# настройка обработчика и форматировщика в соответствии с нашими нуждами
long_handler = logging.FileHandler("data/long_logger.log", encoding='utf-8', mode='w')
long_formatter = logging.Formatter("%(asctime)s %(levelname)s %(funcName)s %(message)s")

# добавление форматировщика к обработчику 
long_handler.setFormatter(long_formatter)
# добавление обработчика к логгеру
long_logger.addHandler(long_handler)

In [18]:
def _save_df_to_zip(df_: pd.DataFrame, archive_name: str = 'archive', folder: str='data', replace: bool=False) -> None:
    # Путь к файлу
    file_path = Path(folder).joinpath(archive_name + '.zip')
    Path(folder).mkdir(exist_ok=True)
    # Проверяем, существует ли файл
    if file_path.exists() and not replace:
        # Получаем время создания файла
        time = datetime.fromtimestamp(file_path.lstat().st_atime).strftime('%Y-%m-%d %H-%M')

        # Создаем новое имя файла с добавлением времени Unix
        new_file_name = file_path.stem + " " + str(time) + file_path.suffix

        # Создаем новый путь для переименованного файла
        new_file_path = file_path.with_name(new_file_name)
        # Переименовываем файл
        file_path.rename(new_file_path)

# to csv
    compression_opts = dict(method='zip', archive_name=f'{archive_name}.csv')
    df_.to_csv(f'{folder}/{archive_name}.zip', index=False, compression=compression_opts, encoding='utf-8')

In [60]:
loc_df = pd.read_csv('data/located_list.zip')
loc_test = loc_df.iloc[6]

In [97]:
ret_df = pd.read_csv('data/retailers.zip')
ret_test = ret_df.iloc[200]
ret_test

date                                2023-09-19 16:58:14.709033
sity                                                   ivanovo
market                                       Созвездие Красоты
discounts                                                  684
data_uuid                 320b1e34-b660-4876-911e-162a1fa9b0a4
href         https://edadeal.ru/ivanovo/retailers/sozvezdie...
Name: 200, dtype: object

In [108]:
segments_list = list()
url = 'https://search.edadeal.io/api/v4/search'
with tqdm(total=ret_df.shape[0]) as pbar:
    for index, item in ret_df.iterrows():
        
        res = loc_df[loc_df['slug'] == item.sity].min()
    
        headers = {
        'x-locality-geoid': str(res.geoId),
        'x-position-latitude': f'{res.lat:.5f}',
        'x-position-longitude': f'{res.lng:.5f}'
        }
        
        params = {
            'groupBy': ['meta'],
             'noContent': ['true'],
             'page': ['0'],
             'retailerUuid': [item.data_uuid]
        }
        
        for _ in range(10):
            try:
                respons = requests.get(url, headers=headers, params=params, verify=False).json()
            except ConnectTimeout:
                continue
            else:
                break
            
        segments_list.append((item.sity, item.data_uuid, respons))
        pbar.update(1)


100%|██████████████████████████████████████████████████████████████████████████████| 5159/5159 [18:22<00:00,  4.68it/s]


In [110]:
with open('data/segments_list.pickle', 'wb') as file:
    pickle.dump(segments_list, file)

In [2]:
with open('data/segments_list.pickle', 'rb') as file:
    segments_list = pickle.load(file)

In [16]:
df = pd.DataFrame()
for sity, market, data in segments_list:
    tmp = pd.DataFrame(data.get('entities').get('segments'))
    tmp.insert(0, 'sity', sity)
    tmp.insert(1, 'market', market)
    df = pd.concat([df, tmp])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 541351 entries, 0 to 38
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   sity        541351 non-null  object 
 1   market      541351 non-null  object 
 2   count       541351 non-null  float64
 3   uuid        541351 non-null  object 
 4   slug        541351 non-null  object 
 5   level       541351 non-null  float64
 6   name        541351 non-null  object 
 7   parentUuid  502400 non-null  object 
dtypes: float64(2), object(6)
memory usage: 37.2+ MB


In [19]:
_save_df_to_zip(df, 'segments_id')
# df.to_excel('data/segments_id.xlsx', index=False)

In [24]:
df['level'] = df['level'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 541351 entries, 0 to 38
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   sity        541351 non-null  object 
 1   market      541351 non-null  object 
 2   count       541351 non-null  float64
 3   uuid        541351 non-null  object 
 4   slug        541351 non-null  object 
 5   level       541351 non-null  int32  
 6   name        541351 non-null  object 
 7   parentUuid  502400 non-null  object 
dtypes: float64(1), int32(1), object(6)
memory usage: 35.1+ MB


In [36]:
lev_00 = df.copy()
lev_00 = lev_00.drop(['sity', 'market', 'count'], axis=1)
lev_00 = lev_00.drop_duplicates()
lev_00.shape

(1025, 5)

In [44]:
lev_03 = lev_00[lev_00['level'] == 3].reset_index(drop=True).copy().drop('level', axis=1)
lev_02 = lev_00[lev_00['level'] == 2].reset_index(drop=True).copy().drop('level', axis=1)
lev_01 = lev_00[lev_00['level'] == 1].reset_index(drop=True).copy().drop('level', axis=1)

In [58]:
lev_1 = lev_03.merge(
                        right=lev_02,
                        how='left',
                        left_on='parentUuid',
                        right_on='uuid',
                        suffixes=(' level 03', ' level 02')
                    )
lev_2 = lev_1.merge(
                        right=lev_01,
                        how='left',
                        left_on='parentUuid level 02',
                        right_on='uuid'
                    ).drop(['parentUuid level 03', 'parentUuid level 02', 'parentUuid'], axis=1)
lev_df = lev_2[[
                    'slug',
                    'name',
                    'uuid',
                    'slug level 02',
                    'name level 02',
                    'uuid level 02',
                    'slug level 03',
                    'name level 03',
                    'uuid level 03',
                ]]

In [59]:
lev_df

,slug,name,uuid,slug level 02,name level 02,uuid level 02,slug level 03,name level 03,uuid level 03
0,food,Продукты,3b31ab10-6311-11e6-849f-52540010b608,dairy,Молочные продукты,3b31b1a8-6311-11e6-849f-52540010b608,milk,Молоко,3b31b275-6311-11e6-849f-52540010b608
1,food,Продукты,3b31ab10-6311-11e6-849f-52540010b608,dairy,Молочные продукты,3b31b1a8-6311-11e6-849f-52540010b608,cultured-milk-foods,Кисломолочные продукты,3b31b5d2-6311-11e6-849f-52540010b608
2,food,Продукты,3b31ab10-6311-11e6-849f-52540010b608,dairy,Молочные продукты,3b31b1a8-6311-11e6-849f-52540010b608,cheese,Сыр,3b31b30e-6311-11e6-849f-52540010b608
3,food,Продукты,3b31ab10-6311-11e6-849f-52540010b608,dairy,Молочные продукты,3b31b1a8-6311-11e6-849f-52540010b608,cream-cheese,Сыр плавленый,3b31b396-6311-11e6-849f-52540010b608
4,food,Продукты,3b31ab10-6311-11e6-849f-52540010b608,dairy,Молочные продукты,3b31b1a8-6311-11e6-849f-52540010b608,maslo-margarin-spred,"Масло, маргарин, спред",3b31b415-6311-11e6-849f-52540010b608
...,...,...,...,...,...,...,...,...,...
823,mkt-electronic-1077,Электроника,4034e4ee-db86-54ab-b0e5-2424261c26ae,mkt-tv-and-audio-appliances-1896,Аудио- и видеотехника,2faf15ce-df3e-5dc8-b85c-956ab83d1e40,mkt-dvd-and-blu-ray-players-561,DVD и Blu-ray плееры,1c924328-6966-5ab2-8838-ca00ce376c95
824,mkt-construction-and-repair-482,Строительство и ремонт,25712009-20fb-554b-8567-e658c7329090,mkt-heating-784,Отопление и вентиляция,fe02bc36-2137-57bd-b0d6-2f279fcc7663,mkt-gazovye-obogrevateli-2908,Газовые обогреватели,db8df00c-acee-5e5a-968f-a63706047f09
825,mkt-leisure-and-entertainment-474,Досуг и развлечения,dc7bc201-e180-5b09-85d4-1bdc8de2e4b0,mkt-musical-instruments-643,Музыкальные инструменты,756085ad-5eb7-57aa-8436-e8711a954f92,mkt-acoustic-piano-648,Акустические пианино,faa9b998-3c6c-500d-bbda-12d7584d9a58
826,mkt-computers-477,Компьютерная техника,67aef637-fa77-59fd-96bb-e7b091b39b30,mkt-equipment-for-presentations-926,Оборудование для презентаций,a4c8d31f-f418-55b3-9a60-2fba0602c8c1,mkt-projector-accessories-1089,Аксессуары для проекторов,defcdb3e-86fd-5b05-820a-62033121ea9e


In [60]:
_save_df_to_zip(lev_df, 'segments')
lev_df.to_excel('data/segments.xlsx', index=False)